# Imports

In [1]:
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from numpy import dot
from numpy.linalg import norm

sys.path.append("..")

from constants import U1BASE_PATH, U1TEST_PATH
from benchmark.data_tramsformation import convert_raw_to_matrix

# Data

In [2]:
train_ratings = convert_raw_to_matrix(U1BASE_PATH)
test_ratings = convert_raw_to_matrix(U1TEST_PATH)

# Model

In [26]:
class ColaborativeModel:
    cos_sim = lambda a, b: dot(a, b) / (norm(a) * norm(b))

    def __init__(self, num_users: int = 30) -> None:
        self.num_users = num_users

    def fit(self, ratings):
        self.ratings = ratings

    def predict(self, user_ratings):
        similarities = np.array([ColaborativeModel.cos_sim(user_ratings, rating) for rating in self.ratings])
        closest_users = list(reversed(np.argsort(similarities)))
        closest_users = closest_users[:self.num_users]
        
        closest_similarities = similarities[closest_users]
        sum_closest_similarities = sum(closest_similarities)

        closest_ratingss = train_ratings[closest_users]

        prediction = np.sum((closest_ratingss * closest_similarities[:, None]), axis=0) / sum_closest_similarities

        return prediction

In [22]:
def count_rmse(true, prediction):
    true_non_null = true != 0

    user_non_null = true[true_non_null]
    fitted_prediction = prediction[true_non_null]

    rmse = mean_squared_error(user_non_null, fitted_prediction, squared=False)

    return rmse

In [31]:
colab_model = ColaborativeModel(num_users=30)
colab_model.fit(train_ratings)

pred = colab_model.predict(test_ratings[0])

count_rmse(test_ratings[0], pred)

2.2517708752064904

In [ ]:
def count_rmse_on_test(model, test):
    for sample in test:
        pred = model.predict(sample)
        count_rmse(sample, )